In [ ]:
%matplotlib inline

import xarray as xr
from os.path import join, abspath
import os
from scipy import quantile

RUN = abspath(os.getenv('RUN', '../models/13-sst_solin_qt_sl/test/'))
MODEL = abspath(os.getenv('RUN', '../models/13-sst_solin_qt_sl/1.pkl'))
DATA = abspath(os.getenv('RUN', '../data/processed/training.nc'))
NGROOT = "/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/"


data_2d_path = join(RUN, 'OUT_2D', '*.nc')
data_3d_path = join(RUN, 'OUT_3D', '*.nc')
data_stat_path = join(RUN, 'OUT_STAT', '*.nc')

print(data_2d_path)
data_2d = xr.open_mfdataset(data_2d_path)
data_3d = xr.open_mfdataset(data_3d_path)
stat = xr.open_mfdataset(data_stat_path)

data_3d['FQTNN'] *= 86400
data_3d['FQTNN'].attrs['units'] = 'g/kg/d'
data_3d['FSLINN'] *= 86400
data_3d['FSLINN'].attrs['units'] = 'K/d'



# Holoviews options
import holoviews as hv
hv.extension('bokeh')

%opts Image.PW[width=500, height=300, colorbar=True](cmap='magma')
%opts Image[width=500, height=300, colorbar=True](cmap='magma') 

# Predicted Heating in SAM simulation

In [ ]:
fqt_im = hv.Dataset(data_3d.FQTNN[1:24, :10:2]).to.image(["x", "y"], label='FQTNN')
fsl_im = hv.Dataset(data_3d.FSLINN[1:24, :10:2]).to.image(["x", "y"], label='FSLINN')

In [ ]:
%%opts Image {+framewise}

(fqt_im + fsl_im).cols(1)

There is striping in the lower level heating and moistening in both simulations. The problem is much worse for the FSLINN, which has noticeble issues for all levels below 1km. On the other hand, FQTNN seems okay above for 395m and above.

# Column Run

Let's see if the column simulation also has the zonal-striping

In [ ]:
import torch
from uwnet.model import MLP
from uwnet.columns import run_column

model = MLP.from_path(MODEL)
ngaqua = xr.open_dataset(DATA).sel(time=slice(100, 110))

This cell takes about 2 minutes to run if the column run has not already been performed.

In [ ]:
column_path = ".tmpcol89398.nc"
if not os.path.isfile(column_path):

    columns_run = run_column(model, ngaqua)
    columns_run.to_netcdf(column_path)
else:
    columns_run = xr.open_dataset(column_path)

## Precipitable Water

In [ ]:
%%opts Image[width=500, height=300, colorbar=True](cmap='magma')

PW = (columns_run.layer_mass * columns_run.QT).sum('z')/1000
PW.name = 'PW'
hv.Dataset(PW[::6]).to.image(["x", "y"]).redim.range(PW=(0, 60))

The narrowing of the moist region also, appears in the column simulation, this can be seen in a plot of the zonal mean:

In [ ]:
%%opts Image[width=500, height=300, colorbar=True](cmap='magma')
hv.Image(PW.mean('x'), kdims=["time", "y"])

That said the stripping is not nearly as bad as before. On the other hand, FSLINN and FQTNN do show some evidence of striping. You can see this by playing around with the sliders below.

## Predicted Heating and Moistening

In [ ]:
%%opts Image [width=550] {+framewise}

sl_im = hv.Dataset(columns_run.FSLINN[::3,:20:3]).to.image(["x", "y"], label='FSLINN')
qt_im = hv.Dataset(columns_run.FQTNN[::3,:20:3]).to.image(["x", "y"], label='FQTNN')
(sl_im + qt_im).cols(1)

The heating rate in has severe striping in the boundary layer. I believe this is because the large-scale forcing computed by SAM also has some issues there.

In [ ]:
hv.Dataset(columns_run.FSLI[0,:10:2])\
.to.image(["x", "y"])\
.layout("z")\
.cols(2)\
.redim.range(FSLI=(-40/86400, 40/86400))

In [ ]:
hv.extension('bokeh')

There is also a problem with the inputs in the boundary layer varying dramatically.

# Comparing methods for Computing the forcing.

The zonal-striping in the original simulation caused by problems with the large-scale temperature forcing in the lowest few levels of the model. Let's compare three methods for computing this large-scale forcing

1. Staggered grid aware divergence calculation
2. Advection form with U,V,W and SLI evaluated averaged over the whole cell.
3. SAM based computation. SAM is initialized with the interface averages of the velocities and cell centered averages of the scalar variables.

In [ ]:
from src.data.compute_forcings_finite_difference import compute_advection_tedency_xarray
from gnl.xarray import centderiv



# Compute advection forcing with staggered form of divergence
FSLI_stag =compute_advection_tedency_xarray(ngaqua.isel(time=slice(0,100)), 'SLI')/86400
FSLI_stag.name = 'FSLI'

# Compute advection forcing using centered differences
ds = xr.open_dataset(join(NGROOT, 'coarse', '3d', 'all.nc')).sel(time=ngaqua.time)
u, v, w = ds.U, ds.V, ds.W
f = ngaqua.SLI
FSLI_cent = -(u * centderiv(f, 'x') + v * centderiv(f, 'y', boundary='nearest')
                        + w * centderiv(f, 'z', boundary='nearest'))
FSLI_cent.name = 'FSLI'


# combine these
plot_me = xr.Dataset({'STAG': FSLI_stag, 'Cent': FSLI_cent.compute(), 'SAM': ngaqua.FSLI})\
.to_array(name='FSLI', dim='Method')

In [ ]:
%%opts Image {+framewise}

def hv_slices(x, name=None, dynamic=True):
    if not x.name:
        x.name = name
    return hv.Dataset(x).to.image(["x", "y"], dynamic=dynamic)


hv_slices(plot_me.sel(z=[37, 1000, 10000], time=[100, 108.12], method='nearest'), dynamic=False).layout('Method').cols(1)

All three methods agree pretty well in the free troposphere, but the centered differences computation is much smoother near the boundary layer. This indicates that we should could use Cent for training purposes. Unfortunately, that SAM has so much striping indicates that our initialization strategy is not working in the boundary layer. Our goal is to make **SAM behave like the centered differences**.